In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
input_path = '/home/xli5/KTH-traces/KTH_data/kth_ngram_removed_rep_APs/'
output_path = "/home/xli5/KTH-traces/KTH_data/kth_ada_trace/ap_coords/"

## AP Indices to Coordinates

In [3]:
ap_to_index_df = pd.read_csv('AP_to_index.txt')
ap_locations_df = pd.read_csv('APlocations.txt')

ap_df = pd.merge(ap_to_index_df, ap_locations_df, on='AP')
ap_df.set_index('Index', inplace=True)
ap_df.drop(['floor', 'AP'], axis=1, inplace=True)
ap_df.to_csv('AP_index_to_coordinates.txt')

In [12]:
'''
This function takes in a file with AP indices and then output a x-y coordinate file
'''
def ap_to_coord(dat_file): #input as a dat file 
    full_input_path = input_path + dat_file + '.dat'
    # Read the .dat file containing the trajectory data
    with open(full_input_path, 'r') as file:
        lines = file.readlines()

    # Initialize an empty list to store the trajectory data for each person
    trajectory_data = []

    # Process each line to obtain the trajectory data for each person
    for line in lines:
        # Split the line into individual building numbers
        ap_numbers = line.strip().split()

        # Append the trajectory data for the current person to the list
        trajectory_data.append(ap_numbers)

    # Convert the list of trajectory data into a DataFrame
    trajectory_df = pd.DataFrame(trajectory_data)


    for row_index, row in trajectory_df.iterrows():
        for col_index, val in row.iteritems():
            if val:
                val = int(val)
                x_coord = ap_df.at[val, 'x_coordinate(m)']
                y_coord = ap_df.at[val, 'y_coordinate(m)']
                # Update df1 in place
                trajectory_df.at[row_index, col_index] = (x_coord, y_coord)
            else:
                # Optionally handle or mark values that don't have a corresponding 'Index' in merged_df
                trajectory_df.at[row_index, col_index] = None
        
    output_file = output_path + dat_file + '_output_coordinates.dat'

    with open(output_file, 'w') as file:
        for index, row in trajectory_df.iterrows():
            file.write(f"#{index}:\n")
            file.write(f">{0}:")
        
            for location_set in row:
                if pd.notnull(location_set):  # Check if the location_set is not NaN
                    x_coordinate, y_coordinate = location_set
                    file.write(f"{x_coordinate},{y_coordinate};")
        
            file.write("\n")


In [13]:
dates = ["03-03-1", "03-03-2", "04-28-1", "04-28-2", "09-15-1", "09-15-2", "12-08-1", "12-08-2"]
for d in dates:
    print("starting", d)
    ap_to_coord(d)

starting 03-03-1
starting 03-03-2
starting 04-28-1
starting 04-28-2
starting 09-15-1
starting 09-15-2
starting 12-08-1
starting 12-08-2


## Building Indices to Coordinates

In [2]:
building_info = {}

with open('coordinates.dat', 'r') as file:
    next(file)
    for line in file:
        building_number, x_coordinate, y_coordinate = line.strip().split(',')
        building_info[int(building_number)] = (x_coordinate, y_coordinate)


In [6]:
'''
This function takes in a file of building numbers and maps to x-y coordinates
'''
def building_to_coord(dat_file): #input as a dat file 

    # Read the .dat file containing the trajectory data
    with open(dat_file, 'r') as file:
        lines = file.readlines()
    
    # Initialize an empty list to store the trajectory data for each person
    trajectory_data = []

    # Process each line to obtain the trajectory data for each person
    for line in lines:
        # Split the line into individual building numbers
        building_numbers = line.strip().split()
    
        # Convert building numbers from strings to integers
        building_numbers = [int(building_number) for building_number in building_numbers]
    
        # Append the trajectory data for the current person to the list
        trajectory_data.append(building_numbers)

    # Convert the list of trajectory data into a DataFrame
    trajectory_df = pd.DataFrame(trajectory_data)
    
    for column in trajectory_df.columns:
        trajectory_df[column] = trajectory_df[column].map(building_info)
        
    output_file = 'output_coordinates.dat'

    with open(output_file, 'w') as file:
        for index, row in trajectory_df.iterrows():
            file.write(f"#{index}:\n")
            file.write(f">{0}:")
        
            for location_set in row:
                if pd.notnull(location_set):  # Check if the location_set is not NaN
                    x_coordinate, y_coordinate = location_set
                    file.write(f"{x_coordinate},{y_coordinate};")
        
            file.write("\n")


In [8]:
building_to_coord("kth_ngram_building_indices/03-03-1.dat")

### Building-level trajectory re-generation

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
'''
This function takes in a file of building numbers and maps to x-y coordinates
'''
def building_to_coord(dat_file, output_file): #input as a dat file 

    # Read the .dat file containing the trajectory data
    with open(dat_file, 'r') as file:
        lines = file.readlines()
    
    # Initialize an empty list to store the trajectory data for each person
    trajectory_data = []

    # Process each line to obtain the trajectory data for each person
    for line in lines:
        # Split the line into individual building numbers
        building_numbers = line.strip().split()
    
        # Convert building numbers from strings to integers
        building_numbers = [int(building_number) for building_number in building_numbers]
    
        # Append the trajectory data for the current person to the list
        trajectory_data.append(building_numbers)

    # Convert the list of trajectory data into a DataFrame
    trajectory_df = pd.DataFrame(trajectory_data)
    
    for column in trajectory_df.columns:
        trajectory_df[column] = trajectory_df[column].map(building_info)

    with open(output_file, 'w') as file:
        for index, row in trajectory_df.iterrows():
            file.write(f"#{index}:\n")
            file.write(f">{0}:")
        
            for location_set in row:
                if pd.notnull(location_set):  # Check if the location_set is not NaN
                    x_coordinate, y_coordinate = location_set
                    file.write(f"{x_coordinate},{y_coordinate};")
        
            file.write("\n")

In [3]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

building_info = {}

with open('coordinates.dat', 'r') as file:
    next(file)
    for line in file:
        building_number, x_coordinate, y_coordinate = line.strip().split(',')
        building_info[int(building_number)] = (x_coordinate, y_coordinate)

input_path = '/home/zyuan1/research/KTH-traces/KTH_data/kth_new/indices/'
output_path = '/home/zyuan1/research/KTH-traces/KTH_data/kth_new/coords/'

dates = ["03-03", "04-28", "09-15", "12-08"]
for d in dates:
    dat_file = input_path + d + '.dat'
    output_file = output_path + d + '.dat'
    print(dat_file)
    building_to_coord(dat_file, output_file)

/home/zyuan1/research/KTH-traces/KTH_data/kth_new/indices/03-03.dat
/home/zyuan1/research/KTH-traces/KTH_data/kth_new/indices/04-28.dat
/home/zyuan1/research/KTH-traces/KTH_data/kth_new/indices/09-15.dat
/home/zyuan1/research/KTH-traces/KTH_data/kth_new/indices/12-08.dat
